## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/Colab Notebooks/0830_TextMining_WordEmbedding/TextMining_WordEmbedding

[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/0830_TextMining_WordEmbedding/TextMining_WordEmbedding'
/content/drive/MyDrive/Colab Notebooks/0830_TextMining_WordEmbedding/TextMining_WordEmbedding


In [ ]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 3s (79.5 kB/s)
Reading package li

In [ ]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [ ]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import numpy as np
import pandas as pd

In [ ]:
# data 불러오기
data_dir = os.path.join(os.getcwd(), 'data')

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])
corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [ ]:
corpus[:50]

['‘내장형 심장감시장치 장착 테이저’',
 '테이저 때문에 죽는 사람은 의외로 많다',
 '‘워싱턴 포스트’ 지의 2015년 보도에 따르면, 경찰에게 테이저로 사격을 당한 후 구속 중 사망하는 사람이 1주에 1명 꼴로 발생한다고 한다',
 '전도성 전기 병기인 테이저로 인한 중상자 대부분은 근육 마비에 의한 낙상으로 인해 생긴다',
 '그러나 증거에 따르면 테이저가 심장 마비를 일으킬 수 있는 것으로 나타났다',
 '현재 여러 연구자들은 테이저에 EKG(심전도) 기기를 추가하면 피의자의 심부담을 측정함으로서 피의자의 심장 위험을 막을 수 있다고 생각한다',
 '웨이크 포리스트 뱁티스트 메디컬 센터의 응급 의학교수인 윌리엄 보즈먼은 지난 2009년 동료들과 함께, 테이저의 안전성을 철저히 검증했다',
 '그들은 테이저에 명중된 사람은 99.75%가 아무 부상이 없거나 경상 이하만을 입는다고 결론지었다',
 '보즈먼 교수는 “테이저는 놀랄 만큼 효율적이고 또한 안전합니다',
 '경찰관들이 휴대하는 총기나 가스 분사기, 경봉, 맨손 제압 등 다른 체포 수단과 비교해 볼 때 그 어떤 것보다도 안전합니다',
 '그러나 세상에 100%는 없습니다',
 '구류 중 돌연사가 꾸준히 일어나는 게 그 증거입니다.”라고 말한다',
 '테이저에 피격된 사람 250만 명 중 1명은 심장에 무리가 가고, 이 중 일부는 결국 그로 인해 죽게 된다',
 '“테이저에 피격된 사람이 의학적으로 문제가 있음을 알게 된다면, 신속하게 응급조치를 취해 그 사람의 생명을 구할 수 있습니다.”',
 '1년에 테이저 때문에 죽는 사람이 정확히 몇 명인지는 확실히 발표된 바가 없지만, 보즈먼과 그의 동료들은 더욱 안전한 테이저를 만들고자 한다',
 '보즈먼은 테이저의 기본 구조가 EKG 기기와 의외로 유사하다는 데 착안했다',
 '두 개의 탐침이 전선을 통해 전자 기기와 연결되어 있는 것이다',
 '보즈먼과 동료인 제이슨 스토피라 박사는 두 기기를 결합하는 작업이 가능할 거라고 생각했다.이들은 미국 사

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

[여러분의 설명]

'Josa','Punctuation', 'Number'는 필요 없을 것이라고 판단
정규화와 어근 추출을 진행

In [ ]:
okt = Okt()

In [ ]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.
def CustomTokenization(sentence):
  stop = ['Josa','Punctuation', 'Number']
  tokenized = []
  for i, j in okt.pos(sentence, stem = True, norm = True):
      if j in stop:
           continue
      tokenized.append(i)
  return tokenized

In [ ]:
CustomTokenization('황진우는 천재다')

['황진우', '천재']

In [ ]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(CustomTokenization(i))

  0%|          | 0/1020 [00:00<?, ?it/s]

# 2번

이제 word2vec 모델을 학습시켜 봅시다!

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]

모든 단어를 포함시키기 위해, min_count는 0으로 두었고, 양옆으로 4개까지만 보아도 충분하다고 보아 window를 4로 두었다

In [ ]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec(size=200, window = 4, alpha = 0.02, min_count=0)

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 3869


In [ ]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 100)

(1563172, 1765600)

In [ ]:
model_no_pretrained.wv.vocab.keys()

dict_keys(['‘', '내장', '형', '심장', '감시', '장치', '장착', '테이저', '때문', '죽다', '사람', '의외로', '많다', '워싱턴', '포스트', '지다', '보도', '따르다', '경찰', '사격', '당한', '후', '구속', '중', '사망', '하다', '주', '명', '꼴', '발생', '전도성', '전기', '병기', '인하다', '중상', '자', '대부분', '근육', '마비', '의하다', '낙상', '생기다', '그러나', '증거', '일으키다', '수', '있다', '것', '나타나다', '현재', '여러', '연구자', '들', 'EKG', '심전도', '기기', '추가', '피의자', '심', '부담', '측정', '함', '위험', '막', '생각', '웨이', '크다', '포리스트', '뱁티스트', '메다', '컬', '센터', '응급', '의학', '교수', '윌리엄', '보즈', '멀다', '지난', '동료', '함께', '안전성', '철저하다', '검증', '그', '명중', '되다', '가다', '아무', '부상', '없다', '경상', '이하', '입다', '결론', '“', '놀라다', '만큼', '효율', '적', '또한', '안전하다', '경찰관', '휴대', '총기', '가스', '분사기', '경봉', '맨손', '제압', '등', '다른', '체포', '수단', '비교', '볼', '때', '어떻다', '세상', '늘다', '구류', '돌연사', '꾸준하다', '일어나다', '게', '이다', '”', '말', '피격', '무리', '이', '일부', '결국', '그로', '문제', '음', '알', '신속하다', '조치', '취해', '생명', '구', '에', '정확하다', '몇', '명인', '확실하다', '발표', '바', '더욱', '만들다', '기본', '구조', '유사하다', '데', '착안', '두', '개', '타다', '침', '전선', '통해', '전자',

In [ ]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_no_pretrained.wv.most_similar('형', topn = 20)

[('식기세척기', 0.8646550178527832),
 ('㎛', 0.861653208732605),
 ('만들어지다', 0.8430765867233276),
 ('망각', 0.8363964557647705),
 ('애니메이션', 0.8320646286010742),
 ('마이크로미터', 0.8271770477294922),
 ('추적', 0.8268876075744629),
 ('모바일', 0.8209502696990967),
 ('합작', 0.8207587003707886),
 ('처럼', 0.8198655247688293),
 ('마케팅', 0.8188328742980957),
 ('변신', 0.8155219554901123),
 ('룸바', 0.813868522644043),
 ('담다', 0.8138586282730103),
 ('의지', 0.8131570816040039),
 ('피우다', 0.8105380535125732),
 ('쯤', 0.8069040775299072),
 ('엔진', 0.8063615560531616),
 ('맞춤', 0.8057680130004883),
 ('우주과학', 0.8056814670562744)]

In [ ]:
model_no_pretrained.wv.most_similar('우주', topn = 20)

[('System', 0.9783391952514648),
 ('프로톤', 0.9745914936065674),
 ('Launch', 0.9643229246139526),
 ('우주기지', 0.9568871259689331),
 ('강력하다', 0.9509915113449097),
 ('대결', 0.9485220313072205),
 ('이세돌', 0.9461060762405396),
 ('페타', 0.9402608871459961),
 ('바둑', 0.9387482404708862),
 ('실어', 0.937085747718811),
 ('오리온', 0.9366313219070435),
 ('Space', 0.9364994764328003),
 ('SLS', 0.9349445104598999),
 ('발사', 0.9348280429840088),
 ('소유스', 0.9332600831985474),
 ('묶다', 0.9264953136444092),
 ('운반', 0.9249688386917114),
 ('바이코누르', 0.9245254993438721),
 ('단', 0.9241471290588379),
 ('회족', 0.9232375621795654)]

In [ ]:
model_no_pretrained.wv.most_similar('백금', topn = 20)

[('촉매', 0.9404439926147461),
 ('가격', 0.928458571434021),
 ('섞다', 0.9201295375823975),
 ('귀금속', 0.9114502668380737),
 ('탄소', 0.9051801562309265),
 ('마음대로', 0.8997458219528198),
 ('하향', 0.8923447132110596),
 ('비싸다', 0.8901969790458679),
 ('식', 0.8872993588447571),
 ('부피', 0.8742951154708862),
 ('숨다', 0.8715716004371643),
 ('저렴하다', 0.865201473236084),
 ('완전하다', 0.863239049911499),
 ('거론', 0.8626238107681274),
 ('혼혈', 0.860375702381134),
 ('배열', 0.8601922988891602),
 ('법', 0.8592067956924438),
 ('방사성', 0.8572958707809448),
 ('마찰', 0.8550690412521362),
 ('상용', 0.8522674441337585)]

In [ ]:
model_no_pretrained.wv.most_similar('쥐', topn = 20)

[('감염', 0.9479274153709412),
 ('수컷', 0.945637583732605),
 ('지', 0.9110803008079529),
 ('바이러스', 0.8995733261108398),
 ('카', 0.8992089033126831),
 ('정모', 0.8835717439651489),
 ('고환', 0.8757269978523254),
 ('생식기관', 0.8597354888916016),
 ('쥐다', 0.8320319056510925),
 ('mg', 0.8065440654754639),
 ('성장하다', 0.8035442233085632),
 ('상부', 0.7963391542434692),
 ('병원', 0.7926322221755981),
 ('생식세포', 0.7795026302337646),
 ('태아', 0.7762337327003479),
 ('다이아몬드', 0.7712799310684204),
 ('내과', 0.7711737155914307),
 ('방문', 0.7703171968460083),
 ('돈', 0.7682161331176758),
 ('띄다', 0.7626879811286926)]

In [ ]:
model_no_pretrained.wv.most_similar('브라질', topn = 20)

[('런던대', 0.9768458604812622),
 ('상파울루', 0.9684314727783203),
 ('흐로닝언', 0.9624068140983582),
 ('중반', 0.9503703117370605),
 ('김경식', 0.9415113925933838),
 ('연구실', 0.9412510395050049),
 ('얹다', 0.9382647275924683),
 ('멜버른', 0.937358558177948),
 ('취리히', 0.9358474612236023),
 ('유전체', 0.9342231750488281),
 ('워싱턴', 0.9337989091873169),
 ('세라', 0.9326590895652771),
 ('국립공원', 0.9319744110107422),
 ('하이파', 0.9319144487380981),
 ('스미스', 0.9295549392700195),
 ('준', 0.9290844202041626),
 ('권태', 0.9280469417572021),
 ('마츠다', 0.9279502034187317),
 ('도쿄대', 0.9276313781738281),
 ('카피바라', 0.9266976714134216)]

In [ ]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
pretrained = Word2Vec.load('ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지']))

# 모델 생성
model_pretrained = Word2Vec(size=200, window = 4, alpha = 0.02, min_count=0)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

Pretrained model의 단어 수(V) : 30185
Pretrained model의 embedding vector의 size(N) : 200
현재 vocabulary size(V) : 3869


In [ ]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

현재 vocabulary size(V) : 31473


In [ ]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 100)

(1739467, 1765600)

In [ ]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_pretrained.wv.most_similar('형', topn = 20)

[('형의', 0.7171669602394104),
 ('형과', 0.6603490710258484),
 ('기성품', 0.5838066339492798),
 ('소셜미디어', 0.568292498588562),
 ('기본형', 0.5505474805831909),
 ('식기세척기', 0.5340096950531006),
 ('개량형', 0.5276956558227539),
 ('양용', 0.5245072841644287),
 ('태의', 0.5239418148994446),
 ('아우디', 0.5162932872772217),
 ('준동', 0.5155336856842041),
 ('용', 0.51048743724823),
 ('분광', 0.5088897943496704),
 ('맞춤', 0.5053021311759949),
 ('신형', 0.5050931572914124),
 ('부수차', 0.5043564438819885),
 ('V', 0.5008711814880371),
 ('구형', 0.4964897632598877),
 ('혈액형', 0.4950396418571472),
 ('쿠모', 0.4946608543395996)]

In [ ]:
model_pretrained.wv.most_similar('우주', topn = 20)

[('실어', 0.7009634971618652),
 ('나르다', 0.6912574768066406),
 ('우주인', 0.6807958483695984),
 ('회족', 0.6503417491912842),
 ('칼미크인', 0.6401576995849609),
 ('우주선', 0.6343327164649963),
 ('강력하다', 0.6338526606559753),
 ('Space', 0.621755838394165),
 ('프로톤', 0.6194396018981934),
 ('Launch', 0.6091517210006714),
 ('소유스', 0.6059085130691528),
 ('스키아파렐리', 0.6051179766654968),
 ('System', 0.603326678276062),
 ('우주기지', 0.5989745259284973),
 ('바이코누르', 0.5980131030082703),
 ('카자흐스탄', 0.59593665599823),
 ('EXoMars', 0.5893324613571167),
 ('SLS', 0.5867592096328735),
 ('MS', 0.5834760665893555),
 ('Khan', 0.5819829702377319)]

In [ ]:
model_pretrained.wv.most_similar('백금', topn = 20)

[('마그네슘', 0.8252630233764648),
 ('산화물', 0.8150069117546082),
 ('합금', 0.8146459460258484),
 ('텅스텐', 0.8081916570663452),
 ('아연', 0.7956059575080872),
 ('티타늄', 0.7848261594772339),
 ('리튬', 0.7708609104156494),
 ('베릴륨', 0.7626898288726807),
 ('금속', 0.759880542755127),
 ('비금속', 0.7580078840255737),
 ('크로뮴', 0.7539867162704468),
 ('규소', 0.7536642551422119),
 ('흑연', 0.7533483505249023),
 ('카드뮴', 0.740118145942688),
 ('팔라듐', 0.7364910840988159),
 ('수은', 0.7357597947120667),
 ('이리듐', 0.7315975427627563),
 ('니켈', 0.73042231798172),
 ('질산', 0.7297154664993286),
 ('라듐', 0.727691113948822)]

In [ ]:
model_pretrained.wv.most_similar('쥐', topn = 20)

[('박쥐', 0.6030824184417725),
 ('두더지', 0.5844464302062988),
 ('땃', 0.5599733591079712),
 ('원숭이', 0.5340867042541504),
 ('큰박쥐', 0.5236645936965942),
 ('발톱', 0.5211710929870605),
 ('꼬리', 0.520563006401062),
 ('개구리', 0.5120455026626587),
 ('주머니', 0.510111391544342),
 ('상어', 0.5070608854293823),
 ('생쥐', 0.5064572095870972),
 ('구세계', 0.506432056427002),
 ('족제비', 0.5053248405456543),
 ('거미', 0.5051877498626709),
 ('나비', 0.501004695892334),
 ('도마뱀', 0.5002725124359131),
 ('애기', 0.500188946723938),
 ('설치류', 0.4978710114955902),
 ('캥거루', 0.4829033315181732),
 ('귀박쥐', 0.4819291830062866)]

In [ ]:
model_pretrained.wv.most_similar('브라질', topn = 20)

[('콜롬비아', 0.6883834600448608),
 ('스페인', 0.6791243553161621),
 ('멕시코', 0.6534214019775391),
 ('페루', 0.6228936910629272),
 ('프랑스', 0.6136360168457031),
 ('세네갈', 0.5949528217315674),
 ('카메룬', 0.5938465595245361),
 ('남아메리카', 0.5841697454452515),
 ('이탈리아', 0.5826846957206726),
 ('과테말라', 0.5730949640274048),
 ('쿠바', 0.572977602481842),
 ('필리핀', 0.5711897611618042),
 ('인도네시아', 0.5708889961242676),
 ('스위스', 0.565260648727417),
 ('터키', 0.5612469911575317),
 ('캐나다', 0.5559364557266235),
 ('독일', 0.5549792051315308),
 ('벨기에', 0.5542647242546082),
 ('러시아', 0.5532892942428589),
 ('노르웨이', 0.5522342920303345)]

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

[여러분의 설명]

'브라질'과 '쥐'에 대한 유사도를 살펴보았을 때, 사전 학습 없이 학습한 모델에 비해 사전 학습이 된 모델이 훨씬 잘 분류함을 볼 수 있었다.

# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]

In [ ]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, [])))
len(unique_text)

3869

In [ ]:
# embedding 결과 가져오기
word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(model_pretrained.wv[i])

In [ ]:
vec = np.array(vector)

In [ ]:
vec.shape

(3869, 200)

In [ ]:
pca = PCA(random_state=0).fit_transform(vector)
pca.shape

(3869, 200)

In [ ]:
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)[:, :2]

In [ ]:
tsne.shape

(3869, 2)

In [ ]:
# 시각화해보세요!
data = go.Scatter(
    x=pca[:,0],
    y=pca[:,1],
    text = unique_text,
    mode='markers',
    marker=dict(
        size=5,
        #color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [ ]:
# 시각화해보세요!
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = unique_text,
    mode='markers',
    marker=dict(
        size=5,
        #color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

In [ ]:
# t-SNE
iter_v = [500, 1000]
perplexity_v = [5, 20, 35,  50]
for i in iter_v:
  for j in perplexity_v:
    tsne = TSNE(n_components = 2, random_state = 0, perplexity = j, learning_rate = 0.05, init = 'random', n_iter = i).fit_transform(vector)

    data = go.Scatter(
      x=tsne[:,0],
      y=tsne[:,1],
      text = unique_text,
      mode='markers',
      marker=dict(
        size=5,
        #color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
    )

    layout = go.Layout(
      autosize=False,
      width=1000,
      height=1000,
      margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
      ),
    )

    fig = go.Figure(data=[data], layout=layout)
    fig.show()

# 시각화



Output hidden; open in https://colab.research.google.com to view.

# 수고하셨습니다!